In [ ]:
import quandl
import pandas as pd
import numpy as np
import sys
from math import sqrt
from bokeh.plotting import figure, show
from bokeh.models import Range1d
from bokeh.io import output_notebook
output_notebook()
from scipy.interpolate import interp1d
from bokeh.models import HoverTool
Loading BokehJS ...

# The list of stocks included in the query
#
stock_list = ['AAPL', 'MCD', 'BK','STZ','JNJ','DIS','MSFT','FB','KO','UNP','NFLX','GS','AMZN','BRK.B']
num_trade_dys_py = 250
num_random_portfolios = 50000

# The quandl api call will return the historical pricing information and stores the data in a table
# The columns I've requested are the ticker, the closing price and the associated date
#
quandl.ApiConfig.api_key = 'ENTER KEY HERE'
raw_data=quandl.get_table("SHARADAR/SEP", qopts={"columns":['ticker','date','close']},ticker=stock_list, paginate=True)

# The data is pivoted so that the column headings are the ticker symbols and the date is the index
#
quandl_data = raw_data.pivot(index='date',columns='ticker', values='close')

# Some quotes are not found in quandl. I needed to download pricing data from yahoo and upload as a csv
#
external_quotes = pd.read_csv('C:/Users/ryanj/python projects/missing_stock_quotes.csv')
external_quotes['date'] = external_quotes['date'].astype('datetime64[ns]')

# Remove the spaces around the column names
#
external_quotes.columns = external_quotes.columns.str.replace(' ', '')

# Change the index to date
#
external_quotes.set_index('date', inplace=True)

# The quote 'VOO' is the S&P index I will use VOO as my proxy for market risk / return
# I will split the dataframe into two pieces; the missing_quotes and the market_portfolio
market_portfolio = external_quotes[['VOO']]
missing_quotes = external_quotes.loc[:, ~external_quotes.columns.isin(['VOO'])]
# Merge the quandl dataframe with the csv file
#
data = pd.merge(quandl_data, missing_quotes, on='date')

# The number of ticker symbols in DataFrame data
#
num_assets = data.shape[1]
# calculate the daily and annual return of the stocks included in the stock list
#
daily_sec_return = data.pct_change()

# Annualize the daily returns
#
annual_sec_return = daily_sec_return.mean() * num_trade_dys_py
annual_sec_return.shape
(23,)
# Calculate the covariance between stocks in the portfolio based on the daily return and then annualize the covariance
# by multiplying by the number of estimated trading days
# covariance will measure the degree that stocks included in the portfolio move in tandem
# negative covaried stocks move in opposite directions while positively covaried stocks move in the same direction
daily_sec_covariance = daily_sec_return.cov()
annual_sec_covariance = daily_sec_covariance * num_trade_dys_py

# Annualize the daily std deviation
#
annual_sec_std_deviation = (daily_sec_return.std()) * np.sqrt(num_trade_dys_py)
annual_sec_covariance.head()

# Generate an array where the number of rows equal num_random_portfolios and where the total number of columns will tie to the number of stocks in the portfolio
# the columns will total 1
#
X = np.random.dirichlet(np.ones(num_assets),size=num_random_portfolios)

# Convert the array to a dataframe and assign the ticker symbol to the column name
#
rand_port_wts = pd.DataFrame(X, columns=data.columns)
rand_port_wts.shape

# multiply the random portfolio weights by the annualized return
#
portfolio_return = rand_port_wts.dot(annual_sec_return)
portfolio_return = pd.DataFrame(portfolio_return, columns=['portfolio_return'])
# Function to pass one row the rate frame to calculation
rand_port_wts.head()


portfolio_std_deviation = pd.DataFrame(rand_port_wts.apply(lambda x: np.sqrt(np.dot(x.T, np.dot(annual_sec_covariance, x))), axis=1),columns=['portfolio_std_dev'])
portfolio_std_deviation.columns[0]
'port_std_dev'
portfolio_data = pd.DataFrame(rand_port_wts)
portfolio_data['return'] = portfolio_return.portfolio_return
portfolio_data['std_dev'] = portfolio_std_deviation.port_std_dev
portfolio_data['sharpe_ratio'] = portfolio_data['return'] / portfolio_data['std_dev']
portfolio_data

portfolio_data.loc[portfolio_data.sharpe_ratio.idxmax()]